# 01-Sample data

In [42]:
import numpy as np
import numpy
import inspect
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func

import clickhouse_util as ch_util
from clickhouse_util import get_clickhouse_create_sql

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import ibis

Create SqlAlchemy engine

In [45]:
ch_url = 'clickhouse://default@10.0.0.2:8123/default'
engine = sa.create_engine(ch_url)

In [46]:
table = 'flight'
flight_null_count = ch_util.get_null_count(ch_url, table)

In [47]:
flight_null_count

{'Year': 0,
 'Month': 0,
 'DayofMonth': 0,
 'DayOfWeek': 0,
 'DepTime': 2302136,
 'CRSDepTime': 0,
 'ArrTime': 2584478,
 'CRSArrTime': 0,
 'UniqueCarrier': 0,
 'FlightNum': 0,
 'TailNum': 37385420,
 'ActualElapsedTime': 2587529,
 'CRSElapsedTime': 26234,
 'AirTime': 39266398,
 'ArrDelay': 2587529,
 'DepDelay': 2302136,
 'Origin': 0,
 'Dest': 0,
 'Distance': 202000,
 'TaxiIn': 37397295,
 'TaxiOut': 37382704,
 'Cancelled': 0,
 'CancellationCode': 122800263,
 'Diverted': 0,
 'CarrierDelay': 89329433,
 'WeatherDelay': 89329433,
 'NASDelay': 89329433,
 'SecurityDelay': 89329433,
 'LateAircraftDelay': 89329433}

Use pandas to read from SqlAlchemy engine

In [50]:
sql = 'select * from flight limit 100000'
df = pd.read_sql(sql, engine)

Get create sql

In [63]:
print(get_clickhouse_create_sql(df, 'flight', ['Year']))

create table flight (
	Year Int64,
	Month Nullable(Int64),
	DayofMonth Nullable(Int64),
	DayOfWeek Nullable(Int64),
	DepTime Nullable(Float64),
	CRSDepTime Nullable(Int64),
	ArrTime Nullable(Float64),
	CRSArrTime Nullable(Int64),
	UniqueCarrier Nullable(String),
	FlightNum Nullable(Int64),
	TailNum Nullable(String),
	ActualElapsedTime Nullable(Float64),
	CRSElapsedTime Nullable(Float64),
	AirTime Nullable(Float64),
	ArrDelay Nullable(Float64),
	DepDelay Nullable(Float64),
	Origin Nullable(String),
	Dest Nullable(String),
	Distance Nullable(Int64),
	TaxiIn Nullable(Float64),
	TaxiOut Nullable(Float64),
	Cancelled Nullable(Int64),
	CancellationCode Nullable(String),
	Diverted Nullable(Int64),
	CarrierDelay Nullable(String),
	WeatherDelay Nullable(String),
	NASDelay Nullable(String),
	SecurityDelay Nullable(String),
	LateAircraftDelay Nullable(String)
)
Engine = MergeTree
Order by Year


In [64]:
metadata = sa.MetaData(bind=engine)
metadata.reflect(only=['flight'])
flight_tbl = metadata.tables['flight']

# cannot access type for nullable types in Clickhouse
for column in flight_tbl.columns:
    print(column.name)
    break

Year


Use SqlAlchemy ORM to query the database

In [65]:
Session = sessionmaker(bind=engine)
session = Session()

qry = session.query(flight_tbl)

for row in session.query(flight_tbl).filter(
        flight_tbl.c.Month == 2, flight_tbl.c.DayofMonth == 29).limit(10):
    print(row)

(1988, 2, 29, 1, 957, 1000, 1054, 1104, 'PI', 894, None, 57, 64, None, -10, -3, 'DCA', 'SYR', 298, None, None, 0, None, 0, None, None, None, None, None)
(1988, 2, 29, 1, 704, 705, 746, 749, 'PI', 894, None, 42, 44, None, -3, -1, 'JAX', 'CHS', 193, None, None, 0, None, 0, None, None, None, None, None)
(1988, 2, 29, 1, 1121, 1125, 1204, 1200, 'PI', 894, None, 43, 35, None, 4, -4, 'SYR', 'BUF', 134, None, None, 0, None, 0, None, None, None, None, None)
(1988, 2, 29, 1, 1624, 1625, 1711, 1710, 'PI', 895, None, 47, 45, None, 1, -1, 'JFK', 'BDL', 106, None, None, 0, None, 0, None, None, None, None, None)
(1988, 2, 29, 1, 2200, 2147, 2230, 2227, 'PI', 896, None, 30, 40, None, 3, 13, 'BWI', 'CHO', 120, None, None, 0, None, 0, None, None, None, None, None)
(1988, 2, 29, 1, 1825, 1820, 1929, 1930, 'PI', 896, None, 64, 70, None, -1, 5, 'LGA', 'ROC', 254, None, None, 0, None, 0, None, None, None, None, None)
(1988, 2, 29, 1, 1954, 2000, 2057, 2104, 'PI', 896, None, 63, 64, None, -7, -6, 'ROC', 'BW

In [66]:
conn = ibis.clickhouse.connect(host='10.0.0.2', port=9000, database='default')

Get metadata for flight table

In [67]:
sql = '''
select database, table, name, type
from system.columns
where database = 'default'
    and table = 'flight'
'''
df2 = pd.read_sql(sql, engine)

In [68]:
df2.head()

,database,table,name,type
0,default,flight,Year,Int16
1,default,flight,Month,Int8
2,default,flight,DayofMonth,Int16
3,default,flight,DayOfWeek,Int8
4,default,flight,DepTime,Nullable(Int16)


In [69]:
table = conn.table('flight')
ibis.options.interactive = True

Get TailNum count and row count

In [70]:
table.TailNum.count()

86149549

In [71]:
table.count()

123534969

Compute percent of null values

In [72]:
df.isna().sum() * 100/df.shape[0]

Year                   0.000
Month                  0.000
DayofMonth             0.000
DayOfWeek              0.000
DepTime                3.812
CRSDepTime             0.000
ArrTime                4.189
CRSArrTime             0.000
UniqueCarrier          0.000
FlightNum              0.000
TailNum               26.272
ActualElapsedTime      4.189
CRSElapsedTime         0.011
AirTime               30.206
ArrDelay               4.189
DepDelay               3.812
Origin                 0.000
Dest                   0.000
Distance               0.000
TaxiIn                26.272
TaxiOut               26.272
Cancelled              0.000
CancellationCode     100.000
Diverted               0.000
CarrierDelay         100.000
WeatherDelay         100.000
NASDelay             100.000
SecurityDelay        100.000
LateAircraftDelay    100.000
dtype: float64